## NOTES

#### RESOURCES

In [ ]:
# https://brennan.io/2016/03/02/logging-in-with-requests/
# https://kazuar.github.io/scraping-tutorial/

In [99]:
import json
import pandas as pd
import csv
import datetime
import collections
import re
import copy

from pprint import pprint

from bs4 import BeautifulSoup, NavigableString, Tag
import lxml.html
import requests
from string import capwords
from titlecase import titlecase
from pathlib2 import Path
import unicodecsv as csv

pprint('all done')

'all done'


#### FOR INDIVIDUAL TESTING

In [227]:
forum = str(65)
page = str(1)
file_name = 'fid_'+forum+'_pg_'+page+'.html'
file_path = '../data/forums/'+forum+'/'+file_name

html_page = open(str(file_path), 'r').read()
thread_page = BeautifulSoup(html_page, 'html.parser')
thread_page = str(thread_page)
no_literals = thread_page.replace('\n', '').replace('\t', '').replace('\r', '')
thread_page = BeautifulSoup(no_literals, 'html.parser') #s kip u'html PUBLIC "-//W3C//

threads = get_threads(thread_page)
# cleaned_threads = clean_threads(threads)
# data = get_thread_data(cleaned_threads)

#### LOAD HTML LIST OF THREADS AND EXTRACT DATA TO CSV

In [382]:
debug = False

if debug:
    in_path = '../examples/forums/'
    out_path = '../examples/threads/'
else:
    in_path = '../data/forums/'
    out_path = '../data/threads/'

forum_paths = [path for path in Path(in_path).iterdir() if (path.is_dir() and path.name != 'logs' and path.name != 'restricted')]

for forum_path in forum_paths:

    forum_id = forum_path.name
    
    num_threads = 0
    
    with open(out_path + 'thread_list_' + forum_id + '.csv', 'a') as csv_file:

        fieldnames = ['forum_id', 'thread_id', 'thread_title', 'num_thread_replies', 'num_thread_views']

        writer = csv.writer(csv_file, delimiter=',')
        writer.writerow(fieldnames)
        
        file_paths = [path for path in forum_path.iterdir() if path.is_file()]

        for file_path in file_paths:

            # LOAD WEBPAGE AND REMOVE STRING LITERALS
            html_page = open(str(file_path), 'r').read()
            thread_page = BeautifulSoup(html_page, 'html.parser')
            thread_page = str(thread_page)
            no_literals = thread_page.replace('\n', '').replace('\t', '').replace('\r', '')
            thread_page = BeautifulSoup(no_literals, 'html.parser') #s kip u'html PUBLIC "-//W3C//
            
            # EXTRACT RELEVANT DATA
            threads = get_threads(thread_page)
            cleaned_threads = clean_threads(threads)
            data = get_thread_data(cleaned_threads)
            
            num_threads += len(data)

            for thread in data:
                row = [forum_id] + thread
                writer.writerow(row)
                
    print 'FINISHED PROCESSING', num_threads, 'THREADS FROM FORUM', forum_id

#### EXTRACT NAV FOR NUMBER OF PAGES

In [215]:
# NOTE: THIS DOES NOT ASSUME EVERY THREAD PAGE WILL HAVE >100 THREADS

def get_num_thread_pages(el):
    
    try:
    
        # THERE IS A TOP AND BOTTOM NAV SO WE'LL JUST TAKE THE TOP
        nav = el.find_all('div', {'class':'pagenav'})[0]
        page_x_of_y = nav.find_all('td', {'class':'vbmenu_control'})[0]
        num_pages = page_x_of_y.text.split(' ')[-1]
        
    except IndexError as er:
        
        num_pages = 1
    
    return num_pages

get_num_thread_pages(thread_page)  #263

#### EXTRACT ALL THREADS, THEN REMOVE 'DIVIDER'

In [216]:
def get_threads(el):
    
    thread = copy.copy(el)
    
    posts = thread.find_all(id=re.compile('threadbits_forum'))
    
    divider = posts[0].find_all('td', {'class' : 'thead'})
    
    if divider:
        
        divider[0].parent.decompose()
    
    # NEED TO REMOVE DIVIDER BETWEEN STICKY POSTS AND REGULAR
#     posts[0].find_all('td', {'class' : 'thead'})[0].parent.decompose()
    
    return posts[0]

threads = get_threads(thread_page)
len(threads) # 103 (since it's the first page, it's 100 + 3 sticky's at the top)

#### REMOVE HTML TABLE COLUMNS 0, 1, AND 3

In [217]:
def clean_threads(threads):
    
    data = []
    
    idx = 0

    for thread in threads:
        
        cols = thread.find_all('td')
        
        cols = [cols[2], cols[4], cols[5]]
            
        data.append(cols)

        idx += 1
    
    return data

cleaned_threads = clean_threads(threads)
len(cleaned_threads)

#### DETECT IF THREAD WAS MOVED, EXTRACT ID, TITLE, REPLIES, AND VIEWS

In [209]:
def get_thread_data(cleaned):
    
    rows = []
    
    for el in cleaned:
        
        thread_id = get_thread_id(el)
        thread_title = get_thread_title(el)
        
        try:
            
            if is_moved_thread(el):
                
                num_thread_replies = 'moved'
                num_thread_views = 'moved'
                
            else:
                
                num_thread_replies = get_thread_replies(el)
                num_thread_views = get_thread_views(el)

            row = [thread_id, thread_title, num_thread_replies, num_thread_views]

            rows.append(row)
    
        except Exception as e:
            
            print e, '-', thread_id + ':', thread_title
        
    return rows

# thread = '../data/forums/645/fid_645_pg_26.hmtl'
# [[u'541001', u'Me n my MWI are celebrating 1year together!!!', u'12', u'676'],
#  [u'540962', u'Question about federal visit form HELP!!!', 'moved', 'moved']]
threads = get_thread_data(cleaned_threads)
print(len(threads))
# threads[72:74]

#### GET THREAD ID

In [263]:
# NOTE: CHANGED B/C ORIGINAL METHOD PICKED UP AN HREF FOR WHICH THE LINK WAS A #

def get_thread_id(el):

    href = el[0].find_all('a', id=re.compile('thread_title'))[0]['href']
    return href.split('=')[-1]

get_thread_id(cleaned_threads[0])# u'77500'

u'77500'

#### GET THREAD TITLE

In [212]:
def get_thread_title(el):
    
    text = el[0].find_all(id=re.compile('thread_title'))[0].text
    # <a href="show... id="thread_title_708682"...">LASO Chit Chat 2018</a>
    return text

get_thread_title(cleaned_threads[-1]) # u'Some tips on prison "etiquette".'

#### DETECT IF THREAD HAS BEEN MOVED

In [211]:
def is_moved_thread(el):
    
    moved = el[0].find_all(string=re.compile('Moved'))
    # <a href="show... id="thread_title_708682"...">LASO Chit Chat 2018</a>
    return bool(moved)

# thread = '../data/forums/645/fid_645_pg_26.hmtl'
is_moved_thread(cleaned_threads[73]) # True

#### GET NO. OF REPLIES TO THREAD

In [213]:
def get_thread_replies(el):
    # <td align='...'><a href="..." onclick="...">540</a></td>
    return el[1].a.text

# get_thread_replies(cleaned_threads[0]) # u'540'

#### GET NO. OF TIMES THREAD HAS BEEN VIEWED

In [214]:
def get_thread_views(el):
    # <td align="center" class="alt2">92,824</td>
    return el[2].text

# get_thread_views(cleaned_threads[0]) # u'92,824'

#### TO DOUBLE CHECK FOR A PREVIOUS ERROR IN WHICH SOME THREAD IDS WERE '#'

In [388]:
for thread_path in thread_paths:
    
    threads = pd.read_csv(thread_path)
    
    fid = threads.at[0, 'forum_id']
    contains_hash = '#' in threads['thread_id'].values
    
    print str(fid) + ':', contains_hash

# 92: False
# 38: False
# 39: False
# 72: False
# 1506: False
# ...